In [1]:
# Aminov Rezo
import numpy as np
import wave
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob
import random

import struct

from keras.models import *
from keras.layers import *
from keras.callbacks import *

import librosa
import soundfile as sf
from keras.models import load_model

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split

/home/aminov/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
DATA_DIR = 'data_v_7_stc'
meta_file = "{}/meta/meta.txt".format(DATA_DIR)
df = pd.read_csv(meta_file, sep='\t',header=None)
labels_name = df[4].unique()

In [11]:
# кодирование лейблов
onehot_dict = {}
for ii, lab in enumerate(labels_name):
    y_ = np.zeros(len(labels_name))
    y_[ii] = 1
    onehot_dict.update({lab:ii})
    
# обратный хэш
hot_to_one = {}
for k,v in onehot_dict.items():
    hot_to_one.update({v:k})

In [10]:
# экстрактор фич: Мел-кепстральные коэффициенты (MFCC). https://habr.com/post/140828/
def extract_feature(file_name):
    X, sample_rate = sf.read(file_name, dtype='float32')
    if X.ndim > 1:
        X = X[:,0]
    X = X.T
    # преобразование Фурье
    stft = np.abs(librosa.stft(X))
    # MFCC
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    # chroma
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    # мэл спектр
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    # спектр-ный контраст
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

In [5]:
files_test = glob(DATA_DIR+'/test/*.wav')

In [6]:
model = load_model('weights/model.hdf5')
model.load_weights('weights/model_weights.hdf5')

In [7]:
CNT=0 # кол-во всех не 'unknown', подмножество 'A'
GOOD=0 # кол-во правильно опред-ых файлов в подмножестве 'A'
BAD=0 # кол-во не правильно опред-ых файлов в подмножестве 'A'

filew = open("result.txt","a") 
features_test = np.empty((0,193))
for file in files_test:    
    try:
        mfccs, chroma, mel, contrast,tonnetz = extract_feature(file)
    except Exception as e:
        print("[Error] extract feature error. %s" % (e))
        continue
    ext_features_test = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
#     features_test = np.vstack([features_test,ext_features_test])
    pred = model.predict(np.expand_dims([ext_features_test],axis=2))
    score = pred.max()
    class_ = hot_to_one[np.argmax(pred)]
    filename = file.split('/')[2]
    
    filew.write(filename+'\t'+str(score)+'\t'+class_+'\n')
    print(filename+' '+str(score)+' '+class_)
    
    # если файл не 'unknown', делаю подсчет совпадений лейбла и наз. файла
    # примерный подсчет, т.к. неизвестно к какому классу относятся файлы
    # с наз. 'unknown'
    if 'unknown' not in filename:
        CNT+=1
        if class_ in filename:
            GOOD+=1
        else:
            BAD+=1
    
filew.close()

unknown_0070.wav 0.8577184 tool
bags_t_0019.wav 0.7882434 bags
door_0085.wav 0.9999981 door
background_0060.wav 0.9987024 background
background_0036.wav 0.9484765 background
speech_0086.wav 0.682044 door
bags_0038.wav 0.9999993 bags
speech_0074.wav 0.97019875 speech
unknown_0034.wav 0.99977094 bags
speech_t_0031.wav 0.99988306 speech
knocking_door_0066.wav 1.0 knocking_door
knocking_door_t_0003.wav 0.9942173 knocking_door
knocking_door_0060.wav 1.0 knocking_door
bags_0086.wav 0.9999908 bags
unknown_0033.wav 0.64536905 door
knocking_door_t_0007.wav 0.96175987 knocking_door
background_t_0011.wav 0.96985525 background
keyboard_t_0001.wav 0.41608772 door
bags_t_0023.wav 0.7920549 bags
unknown_0071.wav 0.9999778 tool
unknown_0089.wav 0.99999905 keyboard
keyboard_0026.wav 0.99999976 keyboard
speech_0008.wav 0.9977386 speech
speech_0006.wav 0.998004 speech
unknown_0115.wav 0.9778926 tool
background_0022.wav 0.8733734 speech
knocking_door_0018.wav 0.90222156 speech
speech_0071.wav 0.958185 spe

unknown_0051.wav 0.63399035 ring
speech_t_0026.wav 0.9999558 speech
background_t_0003.wav 0.9627784 background
speech_0060.wav 0.98575354 speech
speech_0012.wav 0.8908671 speech
speech_t_0004.wav 0.99999774 speech
unknown_0040.wav 0.92507523 keyboard
speech_0038.wav 0.9992618 speech
knocking_door_t_0015.wav 0.52991116 knocking_door
keyboard_t_0011.wav 0.5244095 keyboard
unknown_0041.wav 0.697878 knocking_door
keyboard_t_0015.wav 0.8065575 keyboard
ring_0002.wav 0.9991572 ring
unknown_0137.wav 0.9780918 tool
background_0063.wav 0.98971575 background
keyboard_0046.wav 0.9999999 keyboard
unknown_0102.wav 0.5704234 keyboard
bags_t_0012.wav 0.9999995 bags
unknown_0075.wav 0.7123184 speech
background_t_0001.wav 0.45720744 background
bags_0082.wav 0.9999974 bags
background_t_0013.wav 0.8562463 background
keyboard_0008.wav 0.999998 keyboard
unknown_0087.wav 0.97961193 door
knocking_door_t_0017.wav 0.9995309 door
ring_0048.wav 0.9999914 ring
door_t_0017.wav 0.6759457 door
speech_0031.wav 0.9997

tool_0012.wav 0.986094 tool
speech_0020.wav 0.95596117 speech
ring_0010.wav 1.0 ring
speech_0046.wav 0.99447215 speech
ring_0042.wav 0.999539 ring
unknown_0094.wav 0.99999964 background
ring_0014.wav 0.9667928 ring
speech_0072.wav 0.9971046 speech
ring_0049.wav 0.99995005 ring
unknown_0037.wav 0.74734056 tool
unknown_0047.wav 0.9896909 tool
speech_t_0020.wav 0.9992867 speech
keyboard_0005.wav 0.99983096 keyboard
knocking_door_0043.wav 0.99985254 door
ring_0038.wav 0.99999726 ring
ring_t_0012.wav 1.0 ring
bags_t_0018.wav 0.72961366 bags
keyboard_0049.wav 0.99999905 keyboard
speech_0081.wav 0.9089064 speech
unknown_0035.wav 0.9999831 tool
background_0031.wav 0.9658016 background
unknown_0078.wav 0.36230054 keyboard
knocking_door_0026.wav 1.0 knocking_door
speech_0095.wav 0.9993093 speech
unknown_0114.wav 1.0 ring
unknown_0073.wav 0.89810437 keyboard
unknown_0106.wav 0.4515781 speech
unknown_0120.wav 0.9999994 tool
speech_t_0037.wav 0.9998441 speech
bags_0017.wav 0.9999554 bags
unknown_00

In [8]:
CNT, GOOD, BAD

(473, 417, 56)

In [9]:
GOOD/CNT

0.8816067653276956